<a href="https://colab.research.google.com/github/pmecwan/MAT-421/blob/main/Project_plan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Introduction to the problem

2. Related work

3. Proposed methodology/ models

4. Experiment setups

5. Expected results